In [5]:
import pandas as pd
import numpy as np

### Read the files into data frames

In [6]:
df1 = pd.read_csv("movie.box_office.txt", sep="\t", header=None, names=['movie_id', 'boxoffice_hit'])
df2 = pd.read_csv("movie.features.txt", sep="\t", header=None, names=['Feature', 'value', 'movie_id'])

### Create a feature "is_john_goodman" that is 1 if John Goodman stars in the movie else 0

In [7]:
df2['is_john_goodman'] = df2['Feature'] == 'John_Goodman'
df2['is_john_goodman'] = df2['is_john_goodman'].astype(int)

### Remove extra columns, drop duplicates and merge the two dataframes with"is_john_goodman" feature and "boxoffice_hits"

In [8]:
df = df2[['movie_id', 'is_john_goodman']]
df_orig = df.drop_duplicates()
df_final = df_orig.merge(df1, how='inner', left_on='movie_id', right_on='movie_id')

### Create the two lists to be passed to the monte carlo permutation test

In [9]:
# List of box office hit value for all movies that star John Goodman
x = df_final[df_final['is_john_goodman']==1]['boxoffice_hit'].values.tolist()
# List of box office hit value for all movies that do not star John Goodman
y = df_final[df_final['is_john_goodman']==0]['boxoffice_hit'].values.tolist()
#Number of times the lables are shuffled
loop = 10000

### Define the Monte Carlo Permutation test function

In [13]:
def monte_calrlo_perm_test(x, y, loop):
    n = len(x)
    k = 0
    diff = np.abs(np.mean(x) - np.mean(y))
    total = np.concatenate([x, y])
    for j in range(loop):
        np.random.shuffle(total)
        k += diff < np.abs(np.mean(total[:n]) - np.mean(total[n:]))
    return k / loop

In [16]:
def monte_calrlo_perm_test(x, y, loop):
    n = len(x)
    k = 0
    diff = (np.mean(x) - np.mean(y))
    total = np.concatenate([x, y])
    for j in range(loop):
        np.random.shuffle(total)
        k += diff < (np.mean(total[:n]) - np.mean(total[n:]))
    return k / loop

### Execute the function with the values derived

In [17]:
monte_calrlo_perm_test(x, y, loop)

0.0025000000000000001

### Results

Yes we can reject the Null hypothesis that "that movies featuring John Goodman have the same proportion of box office hits as those that do not" with a significance level of alpha = .01 because the p value that we get after running the monte carlo permutation tets os .0046.

